# M10 Assignment Webscraping with seleinum
  Aravind Raju 

In [1]:
#importing libraries
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
####
#Author: brandon chiazza
#version 1.0
#references: 
#https://www.programiz.com/python-programming/working-csv-files
#https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
#https://realpython.com/python-boto3-aws-s3/
#CLI aws s3api create-bucket --bucket my-bucket-name --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2
#https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 
##

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize Chrome WebDriver
s=Service(ChromeDriverManager().install())
browser =  webdriver.Chrome(service=s)

# Load the webpage
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

# Wait for the input element to be clickable
input_element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]')))

# Input data or interact with elements
input_element.send_keys('0')

# Click the search button
search_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]')))
search_button.click()

# Wait for the table to be present
table = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'table.Bordered')))


print(table)


<selenium.webdriver.remote.webelement.WebElement (session="f60aee05146fb4d9a0fc28188dc125b3", element="f.105B2F3CE91670536E86728C602E429B.d.AA27669F48DD1A56FF14AC91E906BFF7.e.92")>


In [3]:
#####CREATE DATE FRAME#####
#create empty dataframe

data = []
for row in table.find_elements(By.CSS_SELECTOR, 'tr')[1:]: #removing the first row cuz of missing /none values
    cols = [cell.text for cell in row.find_elements(By.CSS_SELECTOR, 'td')]
    data.append(cols)

# Create DataFrame, ensuring column names match the expected number of columns
df = pd.DataFrame(data, columns=["Organization Name", "NY Reg #", "EIN", "Registrant Type", "City", "State"])


display(df)


,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
1,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
2,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
3,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
4,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
5,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
6,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
7,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
8,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
9,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA


In [4]:
import creds

In [5]:
import boto3
import time


# Create an S3 client

client_s3 = boto3.client('s3',
                  aws_access_key_id=creds.access_key,
                  aws_secret_access_key=creds.secret_key)

filename= 'group4' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s"%(filename,datetime)

# Convert DataFrame to CSV string
csv_buffer = df.to_csv(index=False)

# Get the filename (optional)
filename = f"{filenames3}.csv"  

# Upload the CSV string to the S3 bucket
client_s3.put_object(Body=csv_buffer, Bucket='m10buucket', Key=filename)

print(f"CSV file uploaded to s3://m10assignment/{filename}!")


CSV file uploaded to s3://m10assignment/group420240409160950.csv!


In [6]:
#Checking
bucket_name = 'm10buucket'

# List objects in the bucket
response = client_s3.list_objects_v2(Bucket=bucket_name)
if 'Contents' in response:
    # Print each object's key (i.e., filename)
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No objects found in the bucket.")

group420240409160950.csv
